In [1]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
import GMModel as gmm
from matplotlib import cm
import sys
from scipy.cluster.vq import kmeans2
from reWeight import *
import itertools
import matplotlib as mpl

In [2]:
# For 1D (non-periodic)
# These fields are mandatory for the program to run. Please edit the user editables (ue) according to your need.

# Read files
inFile = '1D_data.npy'                # input file (ue) rows ~ time-points, columns ~ features
outFile = 'FEL_1D_data.npy'           # output file (ue) last column ~ Free energy, other columns ~ corresponding CV value
MIX = np.load(inFile)

# Train the gmm
fitM = gmm.LearnFEL(MIX)
fitM.trigger()

CV_range = [-1,1]                     # boundaries of CV (ue)
XX=np.linspace(*CV_range,100)              
meshX = XX.reshape(-1,1)

# re-weight the global partition function after GMM fit (OPTIONAL)
# This step is optional when the input data is already reweighted by random sampling
 
# Currently we offer two types of reweighting:
#   1. based on k_means clustering (suggested for data that has zero overlap among states)
#   2. based on random forest classifier (very low non-zero overlaps could be allowed)

reweight = False                # User-defined, please change according to your need
PX = fitM.getPosterior(meshX)
if reweight==True:
    PGX = fitM.bestMODEL.predict_proba(meshX)
    PX = PX/np.sum(PX)
    PXG = PGX.T*PX
    PXG = PXG.T/np.sum(PXG,axis=1)
    means = fitM.bestMODEL.means_
    wts = fitM.bestMODEL.weights_
    # example of reweighting with k_means_clustering
    rwtPX = rewt(data=PXG,means=means,weights=wts,method='k_means_cluster',num_clusters=3)    # Uncomment to use

    # example of reweighting with random forest
    # this function needs a supervided training set, where each metastable state is separately marked with integers
    # labels is an argument where we input the class array for the input dataset
    #LBL = np.repeat(np.arange(0,3,1),150000)        # Class labels | Uncomment to use
    # bias is the argument where we input the weights of each state (in the order of classes)
    #BIAS = [0.88,0.06,0.06]                         # Weights of metastable states | Uncomment to use
    #rwtPX = rewt(data=PXG,means=means,weights=wts,method='random_forest',features=MIX,labels=LBL,bias=BIAS)     # Uncomment to use

elif reweight==False:
    rwtPX = PX
    
kB = 8.314/(4.2*1000)       # Boltzmann constant in unit kcal/mol/K
T = 300                     # Temperature in Kelvin (K)
kT = kB*T
FE = -kT*np.log(rwtPX)
FE = FE - np.min(FE)

MAT = np.array([XX,FE]).T
np.save(outFile,MAT)

In [17]:
# For 2D (non-periodic)
# These fields are mandatory for the program to run. Please edit the user editables (ue) according to your need.

# Read files
inFile = '2D_data_a.npy'              # input file (ue) rows ~ time-points, columns ~ features
outFile = 'FEL_2D_data_a.npy'         # output file (ue) last column ~ Free energy, other columns ~ corresponding CV value
MIX = np.load(inFile)

# Train the gmm
fitM = gmm.LearnFEL(MIX)
fitM.trigger()

CV1_range = [0,18]                    # boundaries of CV1 (ue)
CV2_range = [0,18]                    # boundaries of CV2 (ue)
XX=np.linspace(*CV1_range,100)
YY=np.linspace(*CV2_range,100)

meshX=np.array(np.meshgrid(XX,YY)).T
meshX = meshX.reshape(meshX.shape[0]**meshX.shape[-1],meshX.shape[-1])

# re-weight the global partition function after GMM fit (OPTIONAL)
# This step is optional when the input data is already reweighted by random sampling
 
# Currently we offer two types of reweighting:
#   1. based on k_means clustering (suggested for data that has zero overlap among states)
#   2. based on random forest classifier (very low non-zero overlaps could be allowed)

reweight = False                # User-defined, please change according to your need
PX = fitM.getPosterior(meshX)
if reweight==True:
    PGX = fitM.bestMODEL.predict_proba(meshX)
    PX = PX/np.sum(PX)
    PXG = PGX.T*PX
    PXG = PXG.T/np.sum(PXG,axis=1)
    means = fitM.bestMODEL.means_
    wts = fitM.bestMODEL.weights_
    # example of reweighting with k_means_clustering
    rwtPX = rewt(data=PXG,means=means,weights=wts,method='k_means_cluster',num_clusters=3)    # Uncomment to use

    # example of reweighting with random forest
    # this function needs a supervided training set, where each metastable state is separately marked with integers
    # labels is an argument where we input the class array for the input dataset
    #LBL = np.repeat(np.arange(0,3,1),150000)        # Class labels | Uncomment to use
    # bias is the argument where we input the weights of each state (in the order of classes)
    #BIAS = [0.88,0.06,0.06]                         # Weights of metastable states | Uncomment to use
    #rwtPX = rewt(data=PXG,means=means,weights=wts,method='random_forest',features=MIX,labels=LBL,bias=BIAS)     # Uncomment to use

elif reweight==False:
    rwtPX = PX
    
# save the FEL
X,Y = meshX.T
kB = 8.314/(4.2*1000)       # Boltzmann constant in unit kcal/mol/K
T = 300                     # Temperature in Kelvin (K)
kT = kB*T
FE = -kT*np.log(rwtPX)
FE = FE - np.min(FE)

MAT = np.vstack((X,Y,FE)).T
np.save(outFile,MAT)        # save the free energy landscape

In [3]:
# For 2D (periodic)
import numpy as np
import os
import vmem2d as vm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
import time
import sys

# Read files
inFile = '2D_data_b.npy'              # input file (ue) rows ~ time-points, columns ~ features
outFile = 'FEL_2D_data_b.npy'         # output file (ue) last column ~ Free energy, other columns ~ corresponding CV value
MIX = np.load(inFile)
numVMF=5                              # no. of element distributions for fitting

fitM = vm.VMmix2D(MIX, numVMF)
fitM.trigger()

print("BIC score: ",fitM.bic())
X=fitM.Xrange
X1,X2=np.meshgrid(X,X)
FE=fitM.getFE()

XX1 = X1.ravel()
XX2 = X2.ravel()
FEL = FE.ravel()
MAT = np.array([XX1,XX2,FEL]).T
np.save(outFile,MAT)                  # save the free energy landscape

BIC score:  28447.829344717782
